# Rotate and Crop Yeast Colony Images for BaQFA Analysis

This notebook is designed to preprocess yeast colony images for Barcode Quantitative Fitness Analysis (BaQFA). It performs the following steps:

1. **Image Preprocessing**:
   - Reads grayscale images of yeast colonies from the specified directory.
   - Detects circular regions (colonies) based on size and circularity thresholds.
   - Rotates and crops the images to align colonies for further analysis.

2. **Input Parameters**:
   - `indir_path`: Path to the directory containing the input images.
   - `num_plates_in_img`: Number of plates in each image.
   - `threshold`: Threshold value for binarization.
   - `min_radius` and `max_radius`: Minimum and maximum radius for colony detection.
   - `circularity_threshold`: Minimum circularity value for colony detection.
   - `crop_offset`: Offset for cropping around detected colonies.

3. **Output**:
   - Preprocessed images are saved in a subdirectory named `rotate_crop` within the input directory.

4. **Dependencies**:
   - Python libraries: `cv2`, `numpy`, `os`, `math`, `subprocess`.
   - Ensure the environment is activated using `mamba activate labopt`.

5. **Usage**:
   - Adjust the parameters in the notebook to match your dataset.
   - Run the notebook step by step to preprocess the images.

6. **Notes**:
   - Verify the detected colony coordinates before proceeding to the next steps.
   - If colony detection fails, adjust the parameters (`min_radius`, `max_radius`, `circularity_threshold`) and rerun the notebook.

This notebook is a critical step in preparing yeast colony images for downstream BaQFA analysis.

In [ ]:

import cv2
import numpy as np
import os
import math
import subprocess

In [ ]:
indir_path = '/mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30'
num_plates_in_img = 2
threshold = 105
min_radius = 50
max_radius = 90
circularity_threshold = 0.75
crop_offset = 150
grid_shape = "6x8"

In [ ]:
def rotate_crop (indir_path, threshold, min_radius, max_radius, circularity_threshold, crop_offset):
    """
    Rotates and crops images in a specified directory based on detected circular regions.

    Parameters:
    -----------
    indir_path : str
        Path to the input directory containing images to process.
    threshold : int
        Threshold value for binary image conversion.
    min_radius : int
        Minimum radius of circles to detect.
    max_radius : int
        Maximum radius of circles to detect.
    circularity_threshold : float
        Minimum circularity value to filter detected circles.
    crop_offset : int
        Offset value for cropping around the detected region.

    Returns:
    --------
    None
        The function saves the rotated and cropped images in a subdirectory named 'rotate_crop'
        within the input directory. If no circles are detected or the user rejects the detected
        coordinates, the function exits without saving any images.

    Notes:
    ------
    - The function processes grayscale images with extensions 'tiff', 'tif', 'TIFF', or 'TIF'.
    - The user is prompted to confirm the detected circle coordinates before proceeding.
    - The rotation angle is determined based on the detected circle positions.
    """

    indir_path = indir_path
    outdir_path = os.path.join(indir_path, 'rotate_crop')
    if not os.path.exists(outdir_path):
        os.mkdir(outdir_path)
    
    imgs_list = []
    imgs_list = [f for f in os.listdir(indir_path) if f.endswith(('tiff', 'tif', 'TIFF', 'TIF'))]
    imgs_list.sort()
    images = []
    for img in imgs_list:
        image_path = os.path.join(indir_path, img)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            images.append(image)
        else:
            print('Failed to read image from plate dir: ' + image_path, flush=True)
            break
    
    _, binary_image = cv2.threshold(images[-1], threshold, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    circle_info = []
    for contour in contours:
        M = cv2.moments(contour)
        cX, cY = None, None
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            radius = int(np.sqrt(M["m00"] / np.pi))
            
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            circularity = 0
        elif perimeter != 0:
            circularity = (4 * np.pi * area) / (perimeter ** 2)
        
        if cX is not None and cY is not None:
            circle_info.append((cX, cY, radius, circularity))

            
    min_radius = min_radius 
    max_radius = max_radius 
    circularity_threshold = circularity_threshold 
    filtered_circles = [(x, y, r, circularity) for x, y, r, circularity in circle_info if (min_radius <= r <= max_radius) and (circularity > circularity_threshold)]
    if filtered_circles == []:
        print('Failed to detect circles, reconsider the parameters: min_radius, max_radius, circularity_threshold')
        return
    

    x_list = []
    y_list = []
    points = []
    circularity_list = []
    for i, (x, y, r, circularity) in enumerate(filtered_circles):
        x_list.append(x)
        y_list.append(y)
        points.append([x, y])
        circularity_list.append(circularity)
        print(f"Circle {i+1}: Center ({x}, {y}), Radius {r}, Circularity {circularity}", flush=True)
    
    print("===============================================")
    print("もし、座標がOKであれば、'y'を入力してください。ダメなら'n'を入力してください。", flush=True)
    get_handan = input()
    if get_handan == 'y':
        print('次のステップに進みます。')
    elif get_handan == 'n':
        print('パラメーターを再調整してください。')
        return
    else:
        print('yかnを入力してください。')
        return

    print("===============================================")
    

    left_up = sorted(points,key=lambda x:x[0]+x[1])[0]
    left_down = sorted(points,key=lambda x:x[0]-x[1])[0]
    right_up = sorted(points,key=lambda x:x[0]-x[1])[-1]
    right_down = sorted(points,key=lambda x:x[0]+x[1])[-1]

    dx = left_up[0] - left_down[0]
    dy = left_up[1] - left_down[1]
    rad = math.atan2(dy, dx)
    deg = math.degrees(rad)

    center = [(left_up[0] + left_down[0] + right_up[0] + right_down[0])/4,
            (left_up[1] + left_down[1] + right_up[1] + right_down[1])/4]

    rotation_matrix = cv2.getRotationMatrix2D(center, angle = deg+90, scale=1.0)

    point_left_up = np.array([left_up[0], left_up[1], 1])
    rotated_left_up = np.dot(rotation_matrix, point_left_up)
    rotated_left_up = (int(rotated_left_up[0]), int(rotated_left_up[1]))

    point_left_down = np.array([left_down[0], left_down[1], 1])
    rotated_left_down = np.dot(rotation_matrix, point_left_down)
    rotated_left_down = (int(rotated_left_down[0]), int(rotated_left_down[1]))

    point_right_up = np.array([right_up[0], right_up[1], 1])
    rotated_right_up = np.dot(rotation_matrix, point_right_up)
    rotated_right_up = (int(rotated_right_up[0]), int(rotated_right_up[1]))

    point_right_down = np.array([right_down[0], right_down[1], 1])
    rotated_right_down = np.dot(rotation_matrix, point_right_down)
    rotated_right_down = (int(rotated_right_down[0]), int(rotated_right_down[1]))


    crop_offset = crop_offset
    x1, y1 = rotated_left_up[0] - crop_offset, rotated_left_up[1] - crop_offset
    x2, y2 = rotated_right_down[0] + crop_offset, rotated_right_down[1] + crop_offset
    angle = deg + 90
    center = center
    scale = 1.0
    
    for i, img in enumerate(images):
        rotated_image = img.copy()
        rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, scale)
        rotated_image = cv2.warpAffine(rotated_image, rotation_matrix, (rotated_image.shape[1], rotated_image.shape[0]))
        rotated_cropped_image = rotated_image[y1:y2, x1:x2]
        basename = os.path.basename(imgs_list[i]).split('.')[0]
        output_path = os.path.join(outdir_path, basename+'.tiff')
        cv2.imwrite(output_path, rotated_cropped_image) 
        print('Success to rotate and crop image: ' + output_path)

In [ ]:
def prep (indir_path, num_plates_in_img, threshold=110, min_radius=30, max_radius=40, circularity=0.8, crop_offset=83):
    """
    Prepares and processes yeast colony images for Barcode Quantitative Fitness Analysis (BaQFA).

    This function performs the following steps:
    1. Creates output directories for each plate.
    2. Reads and preprocesses grayscale images from the specified directory.
    3. Splits each image into smaller sections corresponding to individual plates.
    4. Applies rotation and cropping to align colonies for further analysis.

    Parameters:
    -----------
    indir_path : str
        Path to the directory containing the input images.
    num_plates_in_img : int
        Number of plates in each image.
    threshold : int, optional
        Threshold value for binary image conversion (default is 110).
    min_radius : int, optional
        Minimum radius of circles to detect (default is 30).
    max_radius : int, optional
        Maximum radius of circles to detect (default is 40).
    circularity : float, optional
        Minimum circularity value to filter detected circles (default is 0.8).
    crop_offset : int, optional
        Offset value for cropping around the detected region (default is 83).

    Returns:
    --------
    None
        The function saves the processed images in subdirectories named 'plate1', 'plate2', etc.,
        within the input directory. Each plate directory contains rotated and cropped images.

    Notes:
    ------
    - The function processes grayscale images with extensions 'tiff', 'tif', 'TIFF', or 'TIF'.
    - Ensure the input directory contains images with the correct format and resolution.
    - The rotation and cropping are performed using the `rotate_crop` function.
    """
    for i in range(num_plates_in_img):
        if not os.path.exists(indir_path + '/plate' + str(i+1)):
            os.mkdir(indir_path + '/plate' + str(i+1))
        
    imgs_list = []
    imgs_list = [f for f in os.listdir(indir_path) if f.endswith(('tiff', 'tif', 'TIFF', 'TIF'))]
    imgs_list.sort()
    images = []
    for img in imgs_list:
        image_path = os.path.join(indir_path, img)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.flip(image, 0)
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE) 
        if image is not None:
            images.append(image)
            print('Success to read image: ' + image_path)
        else:
            print('Failed to read image: ' + image_path)
            break
    
    y_top = int(0)
    y_middle = int(7016 / 2)
    y_bottom = int(7016)
    
    x_left = int(0)
    x_middle = int(5096 / 2)
    x_right = int(5096)
    
    for i, image in enumerate(images):
        image_for_plate1 = image[x_middle:x_right, y_top:y_middle]
        image_for_plate2 = image[x_left:x_middle, y_top:y_middle]
        image_for_plate3 = image[x_middle:x_right, y_middle:y_bottom]
        image_for_plate4 = image[x_left:x_middle, y_middle:y_bottom]
        tmp_list = [image_for_plate1, image_for_plate2, image_for_plate3, image_for_plate4]
        for i in range(num_plates_in_img):
            basename = os.path.basename(imgs_list[i]).split('.')[0] + '.tiff'
            output_path = os.path.join(indir_path + '/plate' + str(i+1), basename + '.tiff')
            cv2.imwrite(output_path, tmp_list[i])
            
            
    for plate in range(num_plates_in_img):
        indir_plate = indir_path + '/plate' + str(plate+1)
        rotate_crop(indir_plate, threshold, min_radius, max_radius, circularity, crop_offset)


<img src='/mnt/d/200_GitHub_Repository/OT2_yeast/src/plates_align_in_scanner.png' width="500">

In [ ]:
for i in range(num_plates_in_img):
    if not os.path.exists(indir_path + '/plate' + str(i+1)):
        os.mkdir(indir_path + '/plate' + str(i+1))
    
imgs_list = []
imgs_list = [f for f in os.listdir(indir_path) if f.endswith(('tiff', 'tif', 'TIFF', 'TIF'))]
imgs_list.sort()
images = []
for img in imgs_list:
    image_path = os.path.join(indir_path, img)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.flip(image, 0)
    image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE) 
    if image is not None:
        images.append(image)
        print('Success to read image: ' + image_path)
    else:
        print('Failed to read image: ' + image_path)
        break

Success to read image: /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/QFA1705484730_2024-01-17_18-45-30.tiff
Success to read image: /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/QFA1705484730_2024-01-17_18-55-30.tiff
Success to read image: /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/QFA1705484730_2024-01-17_19-05-30.tiff
Success to read image: /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/QFA1705484730_2024-01-17_19-15-30.tiff
Success to read image: /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/QFA1705484730_2024-01-17_19-25-30.tiff
Success to read image: /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/QFA1705484730_2024-01-17_19-35-30.tiff
Success to read image: /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/QFA1705484730_2024-01-17_19-45-30.tiff
Success to read image: /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/QFA1705484730_2024-01-17_19-55-30.tiff
Success to read image: /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/QFA1705484730_2024-01-17_20-05-30.tiff
Success to read image: /mnt/g/qfa/QFA

In [ ]:
y_top = int(0)
y_middle = int(7016 / 2)
y_bottom = int(7016)

x_left = int(0)
x_middle = int(5096 / 2)
x_right = int(5096)

for i, image in enumerate(images):
    image_for_plate1 = image[x_middle:x_right, y_top:y_middle]
    basename_for_plate1 = os.path.basename(imgs_list[i])
    outout_path_for_plate1 = os.path.join(indir_path + '/plate1', basename_for_plate1)
    cv2.imwrite(outout_path_for_plate1, image_for_plate1)
    
    image_for_plate2 = image[x_left:x_middle, y_top:y_middle]
    basename_for_plate2 = os.path.basename(imgs_list[i])
    outout_path_for_plate2 = os.path.join(indir_path + '/plate2', basename_for_plate2)
    cv2.imwrite(outout_path_for_plate2, image_for_plate2)
    



In [ ]:
for plate in range(num_plates_in_img):
    indir_plate = indir_path + '/plate' + str(plate+1)
    rotate_crop(indir_plate, threshold, min_radius, max_radius, circularity_threshold, crop_offset)

Circle 1: Center (1793, 1742), Radius 81, Circularity 0.7721144187394191
Circle 2: Center (1157, 1741), Radius 81, Circularity 0.8036079758964928
Circle 3: Center (1377, 1737), Radius 81, Circularity 0.8000723165443852
Circle 4: Center (1587, 1730), Radius 80, Circularity 0.7832860591415998
Circle 5: Center (735, 1730), Radius 88, Circularity 0.7858194022102748
Circle 6: Center (2231, 1725), Radius 85, Circularity 0.789726285507688
Circle 7: Center (1582, 1526), Radius 75, Circularity 0.8245750730572103
Circle 8: Center (1379, 1524), Radius 75, Circularity 0.8228511948439631
Circle 9: Center (1800, 1517), Radius 73, Circularity 0.7904177500885059
Circle 10: Center (1157, 1519), Radius 76, Circularity 0.834147503349587
Circle 11: Center (2015, 1514), Radius 75, Circularity 0.7879490301841593
Circle 12: Center (940, 1514), Radius 77, Circularity 0.8001548886206691
Circle 13: Center (2233, 1516), Radius 82, Circularity 0.807109045852295
Circle 14: Center (731, 1516), Radius 85, Circularit

In [6]:
for plate in range(num_plates_in_img):
    indir_path_plate = indir_path + '/plate' + str(plate+1) + '/rotate_crop'
    subprocess.call(f"bacolonyzer analyse -d {indir_path_plate} -g {grid_shape} ", shell=True)


Summary of inputs:
Grid:
Expecting 6 rows and 8 columns on plate.
Searching for colony locations automatically.
Assuming that grid occupies at least 80.0%
Corrections:
Lighting correction turned on.
Adaptive segmentation for low contrasts turned off.
Analysis:
Analysing the entire set of images in series.
Reference image not provided.
Outputs will be saved in /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/plate1/rotate_crop/Output_Images & /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/plate1/rotate_crop/Output_Data

Starting analysis:
Earliest image: /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/plate1/rotate_crop/QFA1705484730_2024-01-17_18-45-30.tiff
Latest image: /mnt/g/qfa/QFA1705484730_2024-01-17_18-45-30/plate1/rotate_crop/QFA1705484730_2024-01-20_18-45-35.tiff

Computing position of the grid
This may take a few seconds...

Fitting grid pattern...: 100%|██████████| 99/99 [00:03<00:00, 27.47it/s]
Analysing each of the images:
Analysis complete for QFA1705484730_2024-01-17_18-45-30.ti